# NASA Earth Data Server의 OPENDAPs로부터 GPM자료의 취득

## 1. 아래의 사이트에서 GPM자료 취득

https://gpm1.gesdisc.eosdis.nasa.gov/dods/

* GPM은 관측시기, 보정정도에 따라서 아래와 자료에 대한 취득이 가능 (금번 연구시 ('22.4.28일기준 5. GPM_3IMERGHHL_06은 '21.6월부터 데이터가 없음)

1: GPM_3IMERGDE_06: GPM Level 3 IMERG V06 Early Daily 0.1 x 0.1 degree Precipitation 

2: GPM_3IMERGDF_06: GPM Level 3 IMERG Final V06 Daily 0.1 x 0.1 degree Precipitation

3: GPM_3IMERGDL_06: GPM Level 3 IMERG V06 Late Daily 0.1 x 0.1 degree Precipitation

4: GPM_3IMERGHHE_06: GPM IMERG Early Precipitation L3 Half Hourly 0.1 by 0.1 degree V06 (GPM_3IMERGHHE)

5: GPM_3IMERGHHL_06: GPM IMERG Late Precipitation L3 Half Hourly 0.1 by 0.1 degree V06 (GPM_3IMERGHHL) 

6: GPM_3IMERGHH_06: GPM IMERG Final Precipitation L3 Half Hourly 0.1 by 0.1 degree V06 (GPM_3IMERGHH) 

7: GPM_3IMERGM_06: GPM Level 3 IMERG Monthly 0.1 by 0.1 degree Precipitation V06

8: GPM_IMERG_LandSeaMask.2: Land/Sea static mask relevant to IMERG precipitation 0.1x0.1 degree V2

9: TRMM_TMPA_LandSeaMask.2: Land/Sea static mask relevant to TMPA precipitation 0.25x0.25 degree V2 

## 2. Install pydap

* 아래의 코드는 다음 Link를 참조하였으며, 아래와 동일한 버전의 pydap을 설치하여야 이용가능함


https://towardsdatascience.com/tapping-nasas-earth-data-via-opendap-apis-and-python-towards-earth-data-engineering-e6531095e8a0

In [ ]:
!pip install git+https://github.com/pydap/pydap.git@fc0dd0e6f180e455dbb68a06a85f84b1eca6754f

## 3. pydap 파이썬 라이브러리를 이용하여 NASA Earth Data Server의 OPENDAPs 서버에 접속

* 위의 9개 datasets 중에서 접속하고자 하는 데이터를 바꾸고 싶다면 'GPM_3IMERGHHL_06'을 9개 중 원하는 datasets으로 변경필요

In [ ]:
from pydap.client import open_url
from pydap.cas.urs import setup_session
import numpy as np 

url = 'https://gpm1.gesdisc.eosdis.nasa.gov/dods/GPM_3IMERGHHL_06'
session = setup_session(username="lukechoi76", password="Bins0418**", check_url = url)
dataset = open_url(url, session=session)

## 4. 강우관측소와 좌표를 Dictionary 타입으로 생성

In [ ]:
locations = {"4001430": [127.34, 35.71], "4001440": [127.19, 35.64], "4001450": [126.97, 35.49], "4003420": [127.11, 35.54], "4007450": [127.13, 34.90], "4007470": [127.24, 35.06], 
             "4007472": [127.07, 34.97], "4007474": [127.09, 35.18], "4009460": [127.22, 34.97], "9000140": [127.10, 35.08], "9000233": [127.01, 34.78], "9000234": [127.14, 34.81]}

In [ ]:
indices = {}
for i, value in enumerate(locations):
    lat = locations[value][1]
    lon = locations[value][0]
    lat_range = [lat-0.05, lat+0.05]      # Latitude extents of each station 
    lon_range = [lon-0.05, lon+0.05]      # Longitude extents of each station
    lat_val = dataset['lat'][:].data
    lon_val = dataset['lon'][:].data
    lat_indice = np.where((lat_val >= lat_range[0]) & 
                          (lat_val <= lat_range[1]))[0]
    lon_indice = np.where((lon_val >= lon_range[0]) & 
                          (lon_val <= lon_range[1]))[0]
    indices.update({value:[lat_indice, lon_indice]})

In [ ]:
indices

## 5. 다운로드할 자료의 시작시간 설정

* event_start_time의 year, month, day 만 수정하여 다운로드할 자료의 시작시간만 설정

In [ ]:
from datetime import datetime, timedelta
min_time = datetime(year=2000, month=6, day=1, hour=0, minute=0)
event_start_time = datetime(year=2021, month=1, day=1, hour=0, minute=0)   # 수정
start_time_index = int((event_start_time - min_time).total_seconds()/(60*30))

## For start time check
start_time_days = dataset['time'][start_time_index].data[0]
start_time_date = datetime(year=1, month=1, day=1) + timedelta(days = start_time_days-2)
print(start_time_date)

## 6. GPM자료 다운로드 및 CSV 파일로 저장

* 본 자료는 한달 다운로드에 약 9분정도 소요되며, 한달이상 설정시 Jupyter가 멈춰버리므로 한달씩 끊어서 다운로드 필요

In [ ]:
%%time
var_name = 'precipcal'
total_dataset = {}
for i, value in enumerate(indices):
    var_dataset = dataset[var_name][
    start_time_index:start_time_index + 48*31,    # 수정
    min(indices[value][0]): max(indices[value][0])+1,
    min(indices[value][1]): max(indices[value][1])+1
    ]
    total_dataset.update({value:var_dataset})

In [ ]:
total_dataset

* csv파일을 저장할 폴더를 생성하고 한달씩 반복적으로 데이터를 다운받을 경우 본 코드는 시행하지 않음

In [ ]:
import os
for code in list(total_dataset.keys()):
    os.mkdir(code)

* 저장할 csv파일을 해당 년월에 맞게 수정하고 아래의 코드를 실행하면 각 관측코드 폴더에 csv파일이 저장됨

In [ ]:
import itertools
import pandas as pd
import os

## Get spatio-temporal dimensions and corresponding data
for code in list(total_dataset.keys()):
    time = total_dataset[code]['time'].data
    lat_column = total_dataset[code]['lat'].data
    lon_column = total_dataset[code]['lon'].data
    var_data = total_dataset[code][var_name].data

    time_column = [event_start_time + timedelta(minutes=30*n) for n in range(len(time))]

    ## Transform pydap model data to dataframe
    map_data = [list(time_column), list(lat_column), list(lon_column)]
    map_tuple_list = list(itertools.product(*map_data))
    base_data = pd.DataFrame(map_tuple_list, columns = ['Time', 'Lat', 'Long'])
    base_data[var_name] = var_data.flatten()
    base_data.to_csv(code+'/GPM_data_SJ_Basin_202101.csv')  # 수정

## 7. 다운받은 데이터를 그래프로 확인

In [ ]:
my_data = pd.read_csv(code+'/GPM_data_SJ_Basin_202101.csv')
my_data

In [ ]:
my_data['precipcal'].plot(figsize=(15,5))

## 8. 월별자료를 하나의 CSV파일로 수정

In [ ]:
from glob import glob

In [ ]:
dataset_list = []
for i in total_dataset:
    lists = sorted( glob(f'./data/{i}/*.csv'))
    dataset_list.append(lists)

* CSV파일로 저장하기

In [ ]:
for i, code in enumerate(total_dataset):
    total_df = pd.DataFrame()
    for j, value in enumerate(dataset_list[i]):
        df = pd.read_csv(value)
        df = df.rename(columns={"Time":"date"})
        df['date'] = pd.DatetimeIndex(df['date']) + timedelta(hours=-9)
        total_df=total_df.append(df)
    total_df = total_df[66:20467]
    total_df.to_csv(code+".csv")